# À propos de ce livret



Recherches liées au développement de cette méthode : 
>Tinghui Zhou, Hanhuai Shan, Arindam Banerjee, Guillermo Sapiro (2012). **Kernelized probabilistic matrix factorization: exploiting graphs and side information.**


La méthode implantés provient de ces sources : 
>[**Dépôt 1**](https://github.com/kastnerkyle/School/blob/master/atpr/matrix_factorization.py) *version python*  
>[**Dépôt 2**](https://github.com/sg3510/al_proj) *version matlab*



**Note sur le cachier**
- Terminer le formatage du document



## Préparation de l'environnement pour l'utilisation

In [42]:
# Utilités
import os
import numpy as np
import scipy.sparse as sp
import scipy.linalg as sl

# Chargement des données
import scipy.io

# Barre de progression
from tqdm.auto import trange
from tqdm import tqdm_notebook, notebook

# Présentation des résultats
import matplotlib.pyplot as plot
import matplotlib.cm as cm

In [43]:
os.chdir("C:/Users/amass/OneDrive/02_Education/02_Maitrise/Cours/PROJET_MAITRISE/application")

# Données

## Chargement des données

In [44]:
# Chargement du tenseur de données
tensor = scipy.io.loadmat('data/Guangzhou-data-set/tensor.mat')
tensor = tensor['tensor']

# Chargement de la matrice aléatoire de données
random_matrix = scipy.io.loadmat('data/Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']

# Chargement du tenseur aléatoire de données
random_tensor = scipy.io.loadmat('data/Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

print(
    "Tensor shape : {:} | Random matrix shape : {:} | Random tensor shape : {:}"
    .format(tensor.shape, random_matrix.shape, random_tensor.shape))

Tensor shape : (214, 61, 144) | Random matrix shape : (214, 61) | Random tensor shape : (214, 61, 144)


## Format des données

In [45]:
petit_donnees = True

# Paramètres du jeu de données tronqué
capteurs = 50
jours = 10
sequences = tensor.shape[2]

matrice = True

if petit_donnees == True:
    tensor = tensor[0:capteurs, 0:jours, 0:sequences]

    random_tensor = random_tensor[0:capteurs, 0:jours, 0:sequences]

print("The tensors shapes are : {:} | {:}".format(tensor.shape,
                                                  random_tensor.shape))

if matrice == True:
    dense_mat = tensor.reshape(
        [tensor.shape[0], tensor.shape[1] * tensor.shape[2]])

    print("The matrix shape is : {:}".format(dense_mat.shape))

The tensors shapes are : (50, 10, 144) | (50, 10, 144)
The matrix shape is : (50, 1440)


## Scénario de manquants

In [46]:
missing_rate = 0.4

random_missing = True

# Revoir le code pour que les deux choix puissent être respectés sur l'ensemble des matrices/ tenseurs
if random_missing == True:
    print("Random missing (RM) scenario")
    binary_mat = (np.round(random_tensor + 0.5 - missing_rate).reshape([
        random_tensor.shape[0], random_tensor.shape[1] * random_tensor.shape[2]
    ]))
    print("\nBinary matrix shape : {:}".format(binary_mat.shape))
else:
    print("Non-random missing (NM) scenario")
    binary_tensor = np.zeros(tensor.shape)
    for i1 in range(tensor.shape[0]):
        for i2 in range(tensor.shape[1]):
            binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 -
                                                missing_rate)
    binary_mat = binary_tensor.reshape([
        binary_tensor.shape[0], binary_tensor.shape[1] * binary_tensor.shape[2]
    ])
    print("\nBinary matrix shape : {:}".format(binary_mat.shape))

sparse_mat = np.multiply(dense_mat, binary_mat)

Random missing (RM) scenario

Binary matrix shape : (50, 1440)


# Modélisation

## Spécification des modèles

In [66]:
def graph_kernel(sparse_matrix, adjacency_width = 5, adjacency_strength = 0.5): # , U, V ??
    """
    
    
    
    """
    # Initialisation de U et V dans le modèle général
    N = A.shape[0]
    M = A.shape[1]
    
    # adjacency_width
    CU = sp.diags([1] * (2 * adjacency_width + 1), range(-adjacency_width, adjacency_width + 1), shape=(N, N)).todense()
    DU = np.diagflat(np.sum(CU, 1))
    CV = sp.diags([1] * (2 * adjacency_width + 1), range(-adjacency_width, adjacency_width + 1),
                  shape=(M, M)).todense()
    DV = np.diagflat(np.sum(CV, 1))
    
    LU = DU - CU
    LV = DV - CV
    
    KU = sl.expm(adjacency_strength * LU)
    KV = sl.expm(adjacency_strength * LV)
    SU = np.linalg.pinv(KU)
    SV = np.linalg.pinv(KV)
    
    return SU, SV

In [67]:
def KPMF(sparse_matrix, rang = 10, iterations = 5, learning_rate = 0.1):
    """
    
    
    
    """
    
    # Initialisation des paramètres
    mat_minus = sparse_matrix
    bin_mat = np.asarray(mat_minus > 0, dtype = np.int) # Identifier les valeurs manquantes
    
        # Normalisation de la matrice (?)
    mat_1dim = np.ravel(A)
    moy_mat = np.mean(mat_1dim)
    mat_norm = mat_minus - moy_mat
    
    N = mat_norm.shape[0]
    M = mat_norm.shape[1]
    U = np.random.randn(N, rang)
    V = np.random.randn(rang, M)
    
    # Trouver les kernels
    SU, SV = graph_kernel(mat_norm)
    
    # Entrainement du modèle
    for r in notebook.tqdm(range(iterations), desc = "Itérations"):
        for i in notebook.tqdm(range(N), desc = "Dimension N"):
            for j in notebook.tqdm(range(M), desc = "Dimension M"):
                # Utilisation des valeurs de la mat_binaire qui sont plus grands que 0
                if bin_mat[i, j] > 0:
                    e = mat_norm[i, j] - np.dot(U[i, :], V[:, j]) # Signifie quoi ?
                    # Imputation de la valeur manquante
                    U[i, :] = U[i, :] + l * (e * V[:, j] - np.dot(SU[i, :], U))
                    V[:, j] = V[:, j] + l * (e * U[i, :] - np.dot(V, SV[:, j]))
    
    # Imputation des valeurs manquantes
    mat_imput = np.dot(U, V)
    
    # Dénormalisation de la matrice
    mat_imput = mat_imput + moy_mat
    
    # Mettre le RMSE à l'intérieur ??
    
    return mat_imput

## Entrainement du modèle

In [70]:
mat_imput1 = KPMF(sparse_mat)

# Présentation des résultats

In [69]:
def get_RMSE(A, A_):
    """
    A : Matrice originale
    A_ : Matrice imputée
    
    
    """
    A1d = np.ravel(A)
    A_1d = np.ravel(A_)
    e = np.mean((A1d - A_1d)**2)
    return np.sqrt(e)

In [71]:
get_RMSE(dense_mat, mat_imput1)

6.015607969621662

In [72]:
def combine(A, A_):
    """
    Fonction qui combine le jeu de données avec des données manquantes et l'imputation 
    
    Paramètres en entrée
        A : Matrice avec des données manquantes
        A_ : Matrice avec des données imputées
    Paramètres en sortie
        out : Matrice complétée 
    
    """
    
    
    out = np.zeros(A.shape)
    N = A.shape[0]
    M = A.shape[1]
    for i in range(N):
        for j in range(M):
            if A[i, j] == 0:
                out[i, j] = A_[i, j]
            else:
                out[i, j] = A[i, j]
    return out

In [74]:
mat_combine = combine(sparse_mat, mat_imput1)

get_RMSE(dense_mat, mat_combine)

3.903757147520465